This file compiles a new csv with every shot taken in the top five leagues

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [11]:
dir1 = ""

# read in teams.csv
filename = dir1 + r"base_data/teams.csv"
df_teams = pd.read_csv(filename)

# read in leagues.csv
filename = dir1 + r"base_data/leagues.csv"
df_leagues = pd.read_csv(filename)

# read in teamStats.csv
filename = dir1 + r"base_data/teamStats.csv"
df_teamStats = pd.read_csv(filename)

# read in fixtures.csv
filename = dir1 + r"base_data/fixtures.csv"
df_fixtures = pd.read_csv(filename)

# read in players.csv
filename = dir1 + r"base_data/players.csv"
df_players = pd.read_csv(filename)

# read in keyEventDescription.csv
filename = dir1 + r"base_data/keyEventDescription.csv"
df_keyEventDescription = pd.read_csv(filename)

C:\Users\caele\AppData\Local\Temp\ipykernel_11088\2743355242.py:21: DtypeWarning: Columns (2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_players = pd.read_csv(filename)


Features we care about: eventId, teamId, playerId, freekick, penalty, x_loc, y_loc, goal?, player position, time?

In [8]:
# only look at teamStats from top five leagues
valid_leagues = ["English Premier League", "French Ligue 1", "Spanish LALIGA", "Italian Serie A", "German Bundesliga"]
valid_leagueIds = list(df_leagues[df_leagues['leagueName'].apply(lambda x: x in valid_leagues)]['leagueId'])
valid_leagueMidsizeNames = list(df_leagues[df_leagues['leagueName'].apply(lambda x: x in valid_leagues)]['midsizeName'])
valid_events = list(df_fixtures[df_fixtures['leagueId'].apply(lambda x: x in valid_leagueIds)]['eventId'])

In [10]:
# load all plays
fname = "plays_data/plays_" + str(2024) + "_" + valid_leagueMidsizeNames[0] + ".csv"
df_plays = pd.read_csv(fname)
for mid_name in valid_leagueMidsizeNames[1:]:
    fname = "plays_data/plays_" + str(2024) + "_" + mid_name + ".csv"
    df_temp = pd.read_csv(fname)
    df_plays = pd.concat([df_plays, df_temp], ignore_index=True)
print(df_plays.shape)
df_plays = df_plays[df_plays['eventId'].apply(lambda x: x in valid_events)]
print(df_plays.shape)
del df_temp

(166191, 23)
(166191, 23)


In [12]:
# load all key events that we want to analyze
fname = "keyEvents_data/keyEvents_" + str(2024) + "_" + valid_leagueMidsizeNames[0] + ".csv"
df_keyEvents = pd.read_csv(fname)
for mid_name in valid_leagueMidsizeNames[1:]:
    fname = "keyEvents_data/keyEvents_" + str(2024) + "_" + mid_name + ".csv"
    df_temp = pd.read_csv(fname)
    df_keyEvents = pd.concat([df_keyEvents, df_temp], ignore_index=True)
print(df_keyEvents.shape)
df_keyEvents = df_keyEvents[df_keyEvents['eventId'].apply(lambda x: x in valid_events)]
print(df_keyEvents.shape)
del df_temp

(48598, 22)
(48598, 22)


In [14]:
df_keyEvents.columns

Index(['seasonType', 'eventId', 'keyEventOrder', 'playId', 'keyEventTypeId',
       'period', 'clockValue', 'clockDisplayValue', 'scoringPlay', 'shootout',
       'keyEventText', 'keyEventShortText', 'teamId', 'goalPositionX',
       'goalPositionY', 'fieldPositionX', 'fieldPositionY', 'fieldPosition2X',
       'fieldPosition2Y', 'participantOrder', 'athleteId', 'updateDateTime'],
      dtype='object')

In [15]:
df_plays.columns

Index(['seasonType', 'eventId', 'playOrder', 'playId', 'typeId', 'text',
       'shortText', 'period', 'clockValue', 'clockDisplayValue', 'teamId',
       'scoringPlay', 'shootout', 'wallclock', 'goalPositionX',
       'goalPositionY', 'fieldpositionX', 'fieldPositionY', 'fieldPosition2X',
       'fieldPosition2Y', 'athleteId', 'participant', 'updateDateTime'],
      dtype='object')